In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
columns = {
    'Сводные данные о продаже наличной иностранной '
    'валюты кредитными организациями физическим лицам (x1)': 'x1',
    'Курс рубля к доллару США (x2)': 'x2',
    'Цена нефти марки "Юралс" (x3)': 'x3',
    'Динамика потребительских цен по группам товаров и услуг '
    '(месяц к соответствующему месяцу предыдущего года,%)  (x4)': 'x4',
    'Объем денежных накоплений в РФ миллиард руб. (y)': 'y'
}

In [6]:
df = pd.read_excel('data/data.xlsx', header=3)
df = df.rename(columns=columns)

,№,Объем денежных накоплений в РФ миллиард руб. (y),Сводные данные о продаже наличной иностранной валюты кредитными организациями физическим лицам (x1),Курс рубля к доллару США (x2),"Цена нефти марки ""Юралс"" (x3)","Динамика потребительских цен по группам товаров и услуг (месяц к соответствующему месяцу предыдущего года,%) (x4)",Unnamed: 6,Unnamed: 7,Задание
0,1,12345.0,2769,29.67,94.2,9.6,NaN,NaN,Постройте линейную модель множественной регрес...
1,2,12091.9,3065,28.94,101.1,9.5,NaN,NaN,Оцените ее качество. Сделайте тесты на значимо...
2,3,12339.7,3899,28.43,111.6,9.5,NaN,NaN,"Постройте парную модель регрессии, для этого в..."
3,4,12450.6,3790,27.50,119.7,9.6,NaN,NaN,Сравните характеристики качества двух моделей....
4,5,12832.1,3963,28.07,112.0,9.6,NaN,NaN,Для выбранной модели рассчитайте бетта и дельт...
5,6,12892.1,4224,28.08,112.0,9.4,NaN,NaN,Проинтерпретируйте полученные результаты.
6,7,13224.5,4645,27.68,115.3,9.0,NaN,NaN,NaN
7,8,13449.9,4914,28.86,109.6,8.2,NaN,NaN,NaN
8,9,13476.8,4830,31.88,112.6,7.2,NaN,NaN,NaN
9,10,13588.9,4815,29.90,108.7,7.2,NaN,NaN,NaN


In [8]:
x = df[columns.keys()]
x

,x1,x2,x3,x4
0,2769,29.67,94.2,9.6
1,3065,28.94,101.1,9.5
2,3899,28.43,111.6,9.5
3,3790,27.50,119.7,9.6
4,3963,28.07,112.0,9.6
5,4224,28.08,112.0,9.4
6,4645,27.68,115.3,9.0
7,4914,28.86,109.6,8.2
8,4830,31.88,112.6,7.2
9,4815,29.90,108.7,7.2


In [10]:
y = df['y']
y

0     12345.0
1     12091.9
2     12339.7
3     12450.6
4     12832.1
5     12892.1
6     13224.5
7     13449.9
8     13476.8
9     13588.9
10    13600.4
11    13768.5
12    14796.6
13    14424.2
14    14607.3
15    14688.0
16    15057.3
17    15161.0
18    15534.6
19    15544.5
20    15644.4
21    15717.8
Name: Объем денежных накоплений в РФ миллиард руб. (y), dtype: float64

# STATSMODELS

In [ ]:
x = sm.add_constant(x)

In [ ]:
model = sm.OLS(y, x).fit()

In [ ]:
predictions = model.predict(x)
predictions

In [ ]:
model.summary()

In [ ]:
# Se
np.sqrt(model.mse_resid)

In [ ]:
# A
np.mean(np.abs(model.resid / y['y'])) * 100
